In [1]:
# Import required libraries

import pandas as pd
import pyodbc
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [2]:
# Download the VADER lexicon for sentiment analysis

nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to C:\Users\Sonu-
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [3]:
# Define a function to connect to database and fetch data using SQL query

def fetch_query():
    # Define the connection string with parameters to connect
    conn_str=(
        "Driver={SQL Server};"
        "Server=SONU-PC;"
        "Database=Marketing_Analytics;"
        "Trusted_Connection=yes;"
    )

    # Establish the connection
    conn = pyodbc.connect(conn_str)

    # Define the sql query
    query = "SELECT ReviewID, CustomerID, ProductID, ReviewDate, Rating, REPLACE(ReviewText,'  ',' ') AS ReviewText FROM dbo.customer_reviews"

    # Fetch data into dataframe from the query
    df = pd.read_sql(query, conn)

    # Close the connection
    conn.close()

    # Return the fetched dataframe
    return df

# Create a datframe and fetch the reviews data using the above defined function
customer_reviews_df = fetch_query()
customer_reviews_df

C:\Users\Sonu-PC\AppData\Local\Temp\ipykernel_34856\2796255994.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText
0,1,77,18,2023-12-23,3,"Average experience, nothing special."
1,2,80,19,2024-12-25,5,The quality is top-notch.
2,3,50,13,2025-01-26,4,Five stars for the quick delivery.
3,4,78,15,2025-04-21,3,"Good quality, but could be cheaper."
4,5,64,2,2023-07-16,3,"Average experience, nothing special."
...,...,...,...,...,...,...
1358,1359,28,4,2023-05-25,3,Not worth the money.
1359,1360,58,12,2023-11-13,2,"Average experience, nothing special."
1360,1361,96,15,2023-03-07,5,Customer support was very helpful.
1361,1362,99,2,2025-12-03,1,Product did not meet my expectations.


In [4]:
# Intitalize the VADER sentiment intensity analyzer

sia = SentimentIntensityAnalyzer()

In [5]:
# Define a function to calculate sentiment scores using VADER

def calculate_sentiment(review):
    # Get sentiment scores of the review text
    sentiment = sia.polarity_scores(review)
    # Return the compound score: Score between -1(Most Negative) and 1(Most Positive)
    return sentiment['compound']

In [6]:
# Define a function to categorize sentiment

def categorize_sentiment(score, rating):
    if score > 0.05:
        if rating >= 4:
            return 'Positive'
        elif rating==3:
            return 'Mixed Positive'
        else:
            return 'Mixed Negative'
    elif score < -0.05:
        if rating <= 2:
            return 'Negative'
        elif rating == 3:
            return 'Mixed Negative'
        else:
            return 'Mixed Positve'
    else:
        if rating >= 4:
            return 'Positive'
        elif rating <= 2:
            return 'Negative'
        else:
            return 'Neutral'

In [7]:
# Define a function to bucket sentiment score into text ranges

def sentiment_bucket(score):
    if score >= 0.5:
        return '0.5 to 1.0'
    elif 0.0 <= score < 0.5:
        return '0.0 to 0.49'
    elif -0.5 < score <= 0.0:
        return '-0.49 to 0.0'
    else:
        return '-1.0 to -0.5'

In [8]:
# Apply sentiment analysis to calculate sentiment scores for each review

customer_reviews_df['SentimentScore'] = customer_reviews_df['ReviewText'].apply(calculate_sentiment)
customer_reviews_df

,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText,SentimentScore
0,1,77,18,2023-12-23,3,"Average experience, nothing special.",-0.3089
1,2,80,19,2024-12-25,5,The quality is top-notch.,0.0000
2,3,50,13,2025-01-26,4,Five stars for the quick delivery.,0.0000
3,4,78,15,2025-04-21,3,"Good quality, but could be cheaper.",0.2382
4,5,64,2,2023-07-16,3,"Average experience, nothing special.",-0.3089
...,...,...,...,...,...,...,...
1358,1359,28,4,2023-05-25,3,Not worth the money.,-0.1695
1359,1360,58,12,2023-11-13,2,"Average experience, nothing special.",-0.3089
1360,1361,96,15,2023-03-07,5,Customer support was very helpful.,0.6997
1361,1362,99,2,2025-12-03,1,Product did not meet my expectations.,0.0000


In [9]:
# Apply sentiment categories using both text and rating

customer_reviews_df['SentimentCategory'] = customer_reviews_df.apply(lambda row:categorize_sentiment(row['SentimentScore'],row['Rating']), axis = 1)
customer_reviews_df

,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText,SentimentScore,SentimentCategory
0,1,77,18,2023-12-23,3,"Average experience, nothing special.",-0.3089,Mixed Negative
1,2,80,19,2024-12-25,5,The quality is top-notch.,0.0000,Positive
2,3,50,13,2025-01-26,4,Five stars for the quick delivery.,0.0000,Positive
3,4,78,15,2025-04-21,3,"Good quality, but could be cheaper.",0.2382,Mixed Positive
4,5,64,2,2023-07-16,3,"Average experience, nothing special.",-0.3089,Mixed Negative
...,...,...,...,...,...,...,...,...
1358,1359,28,4,2023-05-25,3,Not worth the money.,-0.1695,Mixed Negative
1359,1360,58,12,2023-11-13,2,"Average experience, nothing special.",-0.3089,Negative
1360,1361,96,15,2023-03-07,5,Customer support was very helpful.,0.6997,Positive
1361,1362,99,2,2025-12-03,1,Product did not meet my expectations.,0.0000,Negative


In [10]:
# Apply sentiment bucket to categorize scores into definite ranges

customer_reviews_df['SentimentBucket'] = customer_reviews_df['SentimentScore'].apply(sentiment_bucket)
customer_reviews_df

,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText,SentimentScore,SentimentCategory,SentimentBucket
0,1,77,18,2023-12-23,3,"Average experience, nothing special.",-0.3089,Mixed Negative,-0.49 to 0.0
1,2,80,19,2024-12-25,5,The quality is top-notch.,0.0000,Positive,0.0 to 0.49
2,3,50,13,2025-01-26,4,Five stars for the quick delivery.,0.0000,Positive,0.0 to 0.49
3,4,78,15,2025-04-21,3,"Good quality, but could be cheaper.",0.2382,Mixed Positive,0.0 to 0.49
4,5,64,2,2023-07-16,3,"Average experience, nothing special.",-0.3089,Mixed Negative,-0.49 to 0.0
...,...,...,...,...,...,...,...,...,...
1358,1359,28,4,2023-05-25,3,Not worth the money.,-0.1695,Mixed Negative,-0.49 to 0.0
1359,1360,58,12,2023-11-13,2,"Average experience, nothing special.",-0.3089,Negative,-0.49 to 0.0
1360,1361,96,15,2023-03-07,5,Customer support was very helpful.,0.6997,Positive,0.5 to 1.0
1361,1362,99,2,2025-12-03,1,Product did not meet my expectations.,0.0000,Negative,0.0 to 0.49


In [11]:
# Export the dataframe to CSV

customer_reviews_df.to_csv('C:\\A into C(06-15-2025_23-59)\\Projects\\Power BI Projects\\Marketing_Analysis_Project\\DataAnalystPortfolioProject_PBI_SQL_Python_MarketingAnalytics\\Customer_Reviews_Sentiment_Analysis.csv', index=False)
print('Success')

Success
